In [ ]:
import pandas as pd
import numpy as np
import logit

In [ ]:
# Read and transform dataset in one go
dataset = pd.read_csv('swissmetro.dat', sep='\t')\
    .query('(PURPOSE == 1 or PURPOSE == 3) and CHOICE != 0')\
    .assign(car_av_sp = lambda df: df.CAR_AV * (df.SP != 0),
            train_av_sp = lambda df: df.TRAIN_AV * (df.SP != 0),
            sm_av = lambda df: df.SM_AV)
dataset.head()

In [ ]:
# This will be the initial starting point.
betas = logit.Betas(asc_car=0, asc_train=0, asc_sm=0, time=0, cost=0)

# First alternative for specifying utilities: dictionary from label to function
utilities = {
    # train
    1: lambda b, d: betas.get('asc_train', b) +
                        betas.get('time', b) * d['TRAIN_TT'] +
                        betas.get('cost', b) * d['TRAIN_CO'] +
                        logit.avail(d['train_av_sp']),
    # SwissMetro
    2: lambda b, d: betas.get('asc_sm', b) +
                        betas.get('time', b) * d['SM_TT'] +
                        betas.get('cost', b) * d['SM_CO'] +
                        logit.avail(d['sm_av']),
    # Car
    3: lambda b, d: betas.get('asc_car', b) +
                        betas.get('time', b) * d['CAR_TT'] +
                        betas.get('cost', b) * d['CAR_CO'] +
                        logit.avail(d['car_av_sp'])
}

# second alternative: using decorator
utilities = logit.Utilities()

@utilities.for_alternative(1)
def util_train(b, d):
    return betas.get('asc_train', b) +\
                        betas.get('time', b) * d['TRAIN_TT'] +\
                        betas.get('cost', b) * d['TRAIN_CO'] +\
                        logit.avail(d['train_av_sp'])

@utilities.for_alternative(2)
def util_sm(b, d):
    return betas.get('asc_sm', b) +\
                        betas.get('time', b) * d['SM_TT'] +\
                        betas.get('cost', b) * d['SM_CO'] +\
                        logit.avail(d['sm_av'])

@utilities.for_alternative(3)
def util_car(b, d):
    return betas.get('asc_car', b) +\
                        betas.get('time', b) * d['CAR_TT'] +\
                        betas.get('cost', b) * d['CAR_CO'] +\
                        logit.avail(d['car_av_sp'])

In [ ]:
estimates = logit.estimate_logit(betas, utilities, dataset.CHOICE, dataset)

In [ ]:
estimates